In [1]:
import matplotlib.pyplot as plt
import numpy as np
import importlib
import extract_gaze_stimulus
import sys 
sys.path.append("/Users/zacharykelly/Documents/MATLAB/projects/lightLogger/raspberry_pi_firmware/utility")
import Pi_util
import importlib
import scipy.io 
from scipy.signal import find_peaks
import matlab.engine
import tqdm 
import warnings
import hdf5storage


In [ ]:
# Generate a playable video
path_to_recording_chunks: str = "/Volumes/T7 Shield/sam_gazecal_106"
output_dir: str = "./"
Pi_util.generate_playable_videos(path_to_recording_chunks, output_dir, apply_digital_gain=True, fill_missing_frames=True, debayer_images=True, pupil_image_rotation_correction=True)

In [2]:
# Load in the frames of the recording 
world_frames: np.ndarray = Pi_util.destruct_video("./sam_gazecal_106/W.avi", is_grayscale=True)
print(world_frames.shape)

(25792, 480, 640)


In [3]:
# Allocate an array of transformed frames 
transformed_world_frames: np.ndarray = np.zeros_like(world_frames)

In [4]:
# Load in the pupil gaze angles 
gaze_angles: np.ndarray = np.nan_to_num(scipy.io.loadmat("./gaze_angles")["gaze_angles"], 0)
print(gaze_angles.shape)


(25717, 4)


In [5]:
# Load in the first chunk of both cameras. This will tell us if one or the other started first 
world_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/world_time_2025DASH10DASH06_09COLON26COLON14DOT693525_chunk_0_metadata.npy")
pupil_start_chunk_metadata: np.array = np.load("/Volumes/T7 Shield/sam_gazecal_106/pupil_time_2025DASH10DASH06_09COLON26COLON15DOT252214_chunk_0_metadata.npy")

In [6]:
# Find the missing number of frames between the two measurements in time 
FPS: float = 120
missing_frames: int = (abs(len(world_frames) - len(gaze_angles)))
missing_time: float = missing_frames / FPS 
print(missing_frames)
print(missing_time)

# Print the start time of both cameras so we can see whcih is first 
print(world_start_chunk_metadata[0, 0] / (10 ** 9))
print(pupil_start_chunk_metadata[0, 0])

# Find the difference explained by this 
start_time_delay_time: float = abs(pupil_start_chunk_metadata[0, 0] - world_start_chunk_metadata[0, 0] / (10 ** 9))
start_time_delay_frames: int = int(np.ceil(start_time_delay_time * FPS))
print(start_time_delay_time)
print(start_time_delay_frames)

# Account for the fact that we measured the pupil camera time wise is 0.005 phase advanced 
pupil_phase_offset_seconds: float = 0.005
start_time_delay_time -= pupil_phase_offset_seconds
start_time_delay_frames -= int(np.ceil(pupil_phase_offset_seconds * FPS))

75
0.625
3639.024468
3639.559819
0.5353509999999915
65


In [7]:
# Start a new MATLAB session
eng: object = matlab.engine.start_matlab()
eng.tbUseProject("lightLoggerAnalysis", nargout=0)


Locating project "lightLoggerAnalysis" within "/Users/zacharykelly/Documents/MATLAB/projects".
  Found at "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis".
Local copy of ToolboxToolbox is up to date.
Updating "ToolboxRegistry".
Already up to date.
Updating "lightLoggerAnalysis".
Already up to date.
Locating project "lightLogger" within "/Users/zacharykelly/Documents/MATLAB/projects".
  Found at "/Users/zacharykelly/Documents/MATLAB/projects/lightLogger".
Updating "lightLogger".
Already up to date.
Updating "combiLEDToolbox".
Already up to date.
Updating "BrainardLabToolbox".
Already up to date.
Updating "WatsonYellott2012_PupilSize".
Already up to date.
Updating "bads".
Already up to date.
Updating "export_fig".
Already up to date.
Updating "Psychtoolbox-3".
Already up to date.
Updating "SilentSubstitutionToolbox".
Already up to date.
Updating "ExampleTestToolbox".
Already up to date.
Resetting path to factory state.
Adding "ToolboxToolbox" to path at "/Users/zachary

In [8]:
# Initialize the paths we will use 
intrinsics_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/intrinsics_calibration.mat"
transformation_path: str = "/Users/zacharykelly/Documents/MATLAB/projects/lightLoggerAnalysis/code/virtual_foveation_wip/perspective_transform"

In [ ]:
# Iterate over the world frames 
frames_for_video: int = (start_time_delay_frames) + (120 * 5) # len(world_frames)
for world_frame_num in tqdm.trange(start_time_delay_frames, frames_for_video):    
    print(f"Processing frame: {world_frame_num+1}/{frames_for_video}", flush=True)

    # Retrieve the pupil frame that corresponds to this world camera frame 
    pupil_frame_num: int = world_frame_num - start_time_delay_frames

    # If we have gone out of bounds for pupil, simply quit 
    if(pupil_frame_num >= len(gaze_angles)):
        warnings.warn(f"{pupil_frame_num} Out of bounds for pupil frame, breaking")
        break 

    # Otherwise, retrieve the gaze angles for this pupil frame 
    # and world frame 
    world_frame: np.ndarray = world_frames[world_frame_num]
    pupil_gaze_angles: np.ndarray = gaze_angles[pupil_frame_num, :2]
    
    # Then feed this as input into MATLAB to generate the correced frame 
    transformed_world_frame: np.ndarray = np.array(eng.coordinateTransformFinal(matlab.double(np.ascontiguousarray(world_frame).astype(np.float64)),
                                                                                intrinsics_path, 
                                                                                transformation_path,
                                                                                matlab.double(np.ascontiguousarray(pupil_gaze_angles[::-1].astype(np.float64))),
                                                                                nargout=1
                                                                               ))
    transformed_world_frame = np.clip(transformed_world_frame * 255, 0, 255).astype(np.uint8)    
    transformed_world_frame = np.rot90(transformed_world_frame, 2)

    
    transformed_world_frames[world_frame_num] = transformed_world_frame


  0%|          | 0/120 [00:00<?, ?it/s]

Processing frame: 65/184


  1%|          | 1/120 [00:01<02:35,  1.30s/it]

Processing frame: 66/184


  2%|▏         | 2/120 [00:02<02:44,  1.39s/it]

...

 98%|█████████▊| 118/120 [02:52<00:02,  1.45s/it]

Processing frame: 183/184


 99%|█████████▉| 119/120 [02:53<00:01,  1.45s/it]

Processing frame: 184/184


100%|██████████| 120/120 [02:55<00:00,  1.46s/it]


In [ ]:
world_frames_spliced: np.ndarray = np.ascontiguousarray(world_frames[start_time_delay_frames:]).astype(np.float64)
gaze_angles_spliced: np.ndarray = np.ascontiguousarray(gaze_angles[:, :2][:, ::-1]).astype(np.float64)

In [ ]:
hdf5storage.write(
    {'world_frames_spliced': world_frames_spliced},
    path='.', filename='world_frames_spliced.mat',
    matlab_compatible=True, store_python_metadata=False
)

hdf5storage.write(
    {'gaze_angles_spliced': gaze_angles_spliced},
    path='.', filename='gaze_angles_spliced.mat',
    matlab_compatible=True, store_python_metadata=False
)

In [ ]:
# Use function with MATLAB parallel for 

world_frames_spliced: np.ndarray = np.ascontiguousarray(world_frames[start_time_delay_frames:]).astype(np.float64)
gaze_angles_spliced: np.ndarray = np.ascontiguousarray(gaze_angles[:, :2][:, ::-1]).astype(np.float64)

world_frames_spliced = matlab.double(np.ascontiguousarray(world_frames_spliced[:100, :, :]))
gaze_angles_spliced = gaze_angles_spliced[:100, :]

eng.eval("p=gcp('nocreate'); if isempty(p), parpool; end", nargout=0)
transformed_frames: np.ndarray = np.array(eng.perspective_transform_w2e(world_frames_spliced, intrinsics_path, transformation_path, gaze_angles_spliced), dtype=np.uint8)

In [ ]:
# Close the MATLAB engine 
eng.close() 

In [ ]:
np.save("VERY_IMPORTANT_WORLD_FRAMES.npy", transformed_world_frames)

In [ ]:
transformed_world_frames = np.load("VERY_IMPORTANT_WORLD_FRAMES.npy")

In [ ]:
plt.imshow(transformed_world_frames[0])
plt.show()

In [ ]:
print(transformed_world_frames.shape)

In [17]:
Pi_util.frames_to_video(transformed_world_frames, output_path="virtually_foveated_video.avi", fps=120)